## This notebook trains random forest models---trained to predict protein stability using biophysical features---to identify Rocklin designs that are unexpectedly unstable. These designs will be subjected to exhaustive single-site mutations to assess if their stability can be rescued and to inform the engineering of new features and/or improve Rosetta's energy function.

In [1]:
import os
import json
import numpy as np
import pandas as pd
import multiprocessing
from sklearn.neighbors import KDTree
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings("ignore")
from sklearn.ensemble import RandomForestRegressor

## Load preprocessed data, train random forest regressors on biophysical features, and analyze trees to identify unexpectedly unstable designs

**Methodology: We rank designs according to a measure of disparity of the design stability scores from those of the other designs which end up in the same leaves of the trees in a random forest model.**
1. **Train a RF regression model on the entire feature space, with at least K neighbors per leaf**
2. **For each tree of the RF model and each design, idenitfy the other designs in the same leaf**
3. **Compute statistics across the entire forest of differences in stability scores between designs in the same leaf**

In [ ]:
# Load precomputed dataframes
topology_group_list = [1,2,3,4]  # Rocklin 2017, HHH, EHEE, EEHEE, HEEH designs
index_to_name = {1: 'HHH', 2: 'EHEE', 3: 'EEHEE', 4:'HEEH'}

group_df_dict = {}
group_feature_dict = {}

for group_index in topology_group_list:
    group_df_file = f'data/processed_data/Rocklin.{index_to_name[group_index]}.ssm_processed_data.csv'
    group_feature_file = f'data/processed_data/Rocklin.{index_to_name[group_index]}.ssm_processed_data.structural_metrics.json'
    
    group_df_dict[group_index] = pd.read_csv(group_df_file, comment='#')                

    with open(group_feature_file, "r") as f:
        group_feature_dict[group_index] = json.load(f)

In [ ]:
num_near_nhbrs = 10     # identify and report this many nearest leaf neighbors in the saved dataframes
n_estimators = 1000
stability_col = 'stabilityscore_cnn_calibrated'
normalization = 'mean-0; std-1 normalization'

def generate_rfr_leaf_nhbr_design_ranks(run_param):
    (min_samples_leaf, max_features, bootstrap, random_state, out_dir, rfr_model, group_index) = run_param
       
    X = group_df_dict[group_index][group_feature_dict[group_index]].values
    y = group_df_dict[group_index][stability_col].values
    rfr_model.fit(X, y)

    # in each tree, find the samples in the same leaf
    samp_leaf_indices = rfr_model.apply(X)

    # initialize statistical summary arrays
    (num_samps, num_trees) = samp_leaf_indices.shape
    leaf_nhbrs = {i: {j: None for j in range(num_trees)} for i in range(num_samps)}

    avg_diff_stab = np.empty((num_samps, num_trees))
    max_diff_stab = np.empty((num_samps, num_trees))
    
    avg_gt_diff_stab = np.empty((num_samps, num_trees))
    avg_lt_diff_stab = np.empty((num_samps, num_trees))
    max_gt_diff_stab = np.empty((num_samps, num_trees))
    max_lt_diff_stab = np.empty((num_samps, num_trees))

    avg_abs_diff_stab = np.empty((num_samps, num_trees))
    max_abs_diff_stab = np.empty((num_samps, num_trees))

    num_nhbrs = np.empty((num_samps, num_trees))
    num_nhbrs_gt = np.empty((num_samps, num_trees))
    num_nhbrs_lt = np.empty((num_samps, num_trees))

    nearest_nhbrs = np.empty(num_samps, dtype=list)

    for i in range(num_samps):
        # for each design compute statistics of disparity in leaf neighbor stability scores, across each tree
        for j in range(num_trees):
            # find the leaf neighbors across all trees of the current sample
            temp_leaf_nhbrs = np.where(samp_leaf_indices[:,j]==samp_leaf_indices[:,j][i])[0]
            # delete the current sample from the list of leaf neighbors
            leaf_nhbrs[i][j] = np.delete(temp_leaf_nhbrs, np.where(temp_leaf_nhbrs==i))

            # compute statistics of difference between stabilities of designs in each leaf, in each tree
            stability_diff = y[i] - y[leaf_nhbrs[i][j]]
            stability_diff_gt = stability_diff[stability_diff < 0]
            stability_diff_lt = stability_diff[stability_diff > 0]

            avg_diff_stab[i,j] = np.mean(stability_diff)
            max_diff_stab[i,j] = stability_diff[np.argmax(np.abs(stability_diff))]

            avg_gt_diff_stab[i,j] = np.mean(stability_diff_gt)
            avg_lt_diff_stab[i,j] = np.mean(stability_diff_lt)

            if len(stability_diff_gt) == 0:
                max_gt_diff_stab[i,j] = np.nan
            else:
                max_gt_diff_stab[i,j] = np.min(stability_diff_gt)

            if len(stability_diff_lt) == 0:
                max_lt_diff_stab[i,j] = np.nan
            else:
                max_lt_diff_stab[i,j] = np.max(stability_diff_lt)

            # compute statistics of *absolute* difference between stabilities of designs in each leaf, in each tree
            abs_stability_diff = np.abs(stability_diff)
            avg_abs_diff_stab[i,j] = np.mean(abs_stability_diff)
            max_abs_diff_stab[i,j] = np.max(abs_stability_diff)

            # compute statistics of leaf to which design belongs, in each tree
            num_nhbrs[i,j] = len(leaf_nhbrs[i][j])
            num_nhbrs_gt[i,j] = len(stability_diff_gt)
            num_nhbrs_lt[i,j] = len(stability_diff_lt)

        # for each design compute the "frequency distance" to the top nearest leaf neighbors
        temp_leaf_nhbrs = np.array([nhbr for nhbrs in leaf_nhbrs[i].values() for nhbr in nhbrs])
        temp_leaf_nhbrs_count = np.bincount(temp_leaf_nhbrs)
        temp_nearest_leaf_nhbrs = np.argsort(temp_leaf_nhbrs_count)[-num_near_nhbrs:]
        temp_nearest_leaf_nhbrs_freq = temp_leaf_nhbrs_count[temp_nearest_leaf_nhbrs]/n_estimators

        nearest_nhbrs[i] = list(zip(group_df_dict[group_index].loc[temp_nearest_leaf_nhbrs,'name'], 
                                    group_df_dict[group_index].loc[temp_nearest_leaf_nhbrs, stability_col], 
                                    temp_nearest_leaf_nhbrs_freq))
        nearest_nhbrs[i].reverse()

    # num_splits = group_df_dict[group_index][group_feature_dict[group_index]].values.shape[0]
    num_splits = 50
    splitter = KFold(n_splits=num_splits, shuffle=True, random_state=random_state)
    splits_list = list(splitter.split(group_df_dict[group_index].index))
    average_training_inds = np.mean([len(split[0]) for split in splits_list])
    
    splitcount=1
    for split in splits_list:
        train_ind = split[0]
        test_ind = split[1]

        X_train = X[train_ind, :]
        y_train = y[train_ind]

        X_test = X[test_ind, :]
        y_test = y[test_ind]

        rfr_model.fit(X_train, y_train)
        y_pred = rfr_model.predict(X_test)

        group_df_dict[group_index].loc[test_ind, 'stability_pred'] = y_pred
        group_df_dict[group_index].loc[test_ind, 'stability_diff'] = y_test-y_pred
        print('finished split %d out of %d' %(splitcount, num_splits))
        splitcount=splitcount+1
        
    # create dataframes
    dist_df_dict[group_index] = pd.DataFrame()
    dist_df_dict[group_index]['dataset'] = group_df_dict[group_index]['dataset']
    dist_df_dict[group_index]['name'] = group_df_dict[group_index]['name']
    dist_df_dict[group_index]['topology'] = group_df_dict[group_index]['topology']
    dist_df_dict[group_index]['group_index'] = group_df_dict[group_index]['group_index']
    dist_df_dict[group_index][stability_col] = group_df_dict[group_index][stability_col]
    dist_df_dict[group_index]['stability_pred'] = group_df_dict[group_index]['stability_pred']
    dist_df_dict[group_index]['stability_diff'] = group_df_dict[group_index]['stability_diff']
    
    # compute average statistics of leaves to which each design belongs, across all trees
    dist_df_dict[group_index]['num_nhbrs'] = num_nhbrs.mean(axis=1)
    dist_df_dict[group_index]['num_nhbrs_gt'] = np.nanmean(num_nhbrs_gt, axis=1)
    dist_df_dict[group_index]['num_nhbrs_lt'] = np.nanmean(num_nhbrs_lt, axis=1)

    # compute average statistics of the differences between stabilities of designs in each leaf, across all trees
    dist_df_dict[group_index]['diff'] = avg_diff_stab.mean(axis=1)
    dist_df_dict[group_index]['gt_diff'] = np.nanmean(avg_gt_diff_stab, axis=1)
    dist_df_dict[group_index]['lt_diff'] = np.nanmean(avg_lt_diff_stab, axis=1)
    dist_df_dict[group_index]['abs_diff'] = avg_abs_diff_stab.mean(axis=1)

    dist_df_dict[group_index]['max_diff'] = max_diff_stab.mean(axis=1)
    dist_df_dict[group_index]['max_gt_diff'] = np.nanmean(max_gt_diff_stab, axis=1)
    dist_df_dict[group_index]['max_lt_diff'] = np.nanmean(max_lt_diff_stab, axis=1)
    dist_df_dict[group_index]['max_abs_diff'] = max_abs_diff_stab.mean(axis=1)

    for k in range(num_near_nhbrs):
        dist_df_dict[group_index]['leaf_nhbr_%d' %k] = [nearest_nhbr[k] for nearest_nhbr in nearest_nhbrs]

    # write dataframes to disk
    dist_df_dict[group_index].sort_values(by='diff', ascending=False, inplace=True)
    dist_df_dict[group_index].index.name = 'index'
    
    num_features = len(group_feature_dict[group_index])
    topology_name = index_to_name[group_index]
    file_name = out_dir + f'Rocklin.{topology_name}.rfrdisparity.v1.{num_features}.csv'
    
    with open(file_name, "w", newline='') as f:
        f.write("# Design random-forest-leaf-neighbors in topology-specific Rosetta feature space\n")
        f.write("# Topology group index given in protein_groupings_by_uw.v1.metadata.csv: %d \n" %group_index)
        f.write("# Feature space: %s\n" % '; '.join(group_feature_dict[group_index]))
        f.write("# Feature space dimension: %d\n" % len(group_feature_dict[group_index]))
        f.write("# Feature space normalization: %s\n" %normalization)
        f.write("# RF minimum samples per leaf: %d\n" %min_samples_leaf)
        f.write("# RF maximum features per tree: %s\n" %max_features)
        f.write("# RF boostrap?: %s\n" %bootstrap)
        f.write("# RF random state: %s\n" %random_state)
        f.write("# Stability score used: %s\n" %stability_col)
        f.write("# Number of designs: %d\n" %len(dist_df_dict[group_index]))
        f.write('# %s: current best experimental stability score for each design\n' % stability_col)
        f.write('# stability_pred: prediction of rfr model trained on %d designs (on average)\n' % average_training_inds)
        f.write('# stability_diff: difference between experimental and predicted stability score\n')
        f.write("# num_nhbrs: average number of leaf-neighbors across all trees\n")
        f.write("# num_nhbrs_gt: average number of leaf-neighbors across all trees with stability score greater than the design\n")
        f.write("# num_nhbrs_lt: average number of leaf-neighbors across all trees with stability score less than the design\n")
        f.write("# diff: average difference between a design's stability and its leaf-neighbors' stabilities averaged across all trees\n")
        f.write("# gt_diff: average difference between a design's stability and those leaf-neighbors' whose stabilities are greater than the design's averaged across all trees\n")
        f.write("# lt_diff: average difference between a design's stability and those leaf-neighbors' whose stabilities are less than the design's averaged across all trees\n")
        f.write("# abs_diff: average absolute difference between a design's stability and its leaf-neighbors' stabilities averaged across all trees\n") 
        f.write("# max_diff: maximum difference between a design's stability and its leaf-neighbors' stabilities averaged across all trees\n")
        f.write("# max_gt_diff: maximum difference between a design's stability and the leaf-neighbors' whose stabilities are greater than the design's averaged across all trees\n")
        f.write("# max_lt_diff: maximum difference between a design's stability and the leaf-neighbors' whose stabilities are less than the design's averaged across all trees\n")
        f.write("# max_abs_diff: maximum absolute difference between a design's stability and its leaf-neighbors' stabilities averaged across all trees\n")
        f.write("# leaf_nhbr_k: the design that is the kth most often a leaf-neighbor with its stability and the fraction of trees in which it is a neighbor\n")
        f.write("# Author: Francis Motta\n")
        f.write("# Index: Rocklin.%s.filtered.v1.index.csv\n" % group_df_dict[group_index]['topology'][0])
        dist_df_dict[group_index].to_csv(f, index=False)
    
    print('Finished %s' %file_name)
        
# compile multiple run parameters into a list of run-parameter tuples
np.random.seed(0)
random_state = np.random.randint(1,100)

run_param_list = []

out_dir = 'data/rfrdisparity/'
if not os.path.isdir(out_dir):
    os.makedirs(out_dir)
                
for min_samples_leaf in [20]:
    for max_features in [0.5]:
        for bootstrap in [False]:
            random_state = random_state + np.random.randint(1,100)

            rfr_model = RandomForestRegressor(min_samples_leaf=min_samples_leaf,
                                              max_features=max_features,
                                              bootstrap=bootstrap,
                                              n_estimators=n_estimators,
                                              max_leaf_nodes=None,
                                              criterion='mse',
                                              max_depth=None,
                                              min_samples_split=2,
                                              oob_score=False,
                                              n_jobs=-1,
                                              warm_start=False,
                                              random_state=random_state)

            dist_df_dict = {}
            for group_index in topology_group_list:
                run_param_list.append((min_samples_leaf, max_features, bootstrap, random_state, out_dir, rfr_model, group_index))

# training and inference for each design topology group
for run_param in run_param_list[2:4]:
    print(f'== Predicting stability of group {run_param[-1]} designs ==')
    generate_rfr_leaf_nhbr_design_ranks(run_param)

## Extract outlier recommendations for SSM

In [2]:
# load the versioned datasets for outlier recommendations
data_dir = 'data/rfrdisparity/'
meta_dir = 'data/metadata/'
stab_dir = 'data/experimental_stability_scores/'

# -------------------------------------------------------------------------------
meta_file = 'Rocklin.metadata.csv'
stab_file = 'Rocklin.v6.experimental_stability_scores.csv'
stab2_file = 'Rocklin_display_vector_2.rd4.experimental_stability_scores.csv'
data_files = {'HHH':'Rocklin.HHH.rfrdisparity.v1.1527.csv',
              'HEEH': 'Rocklin.HEEH.rfrdisparity.v1.1629.csv',
              'EHEE': 'Rocklin.EHEE.rfrdisparity.v1.1548.csv',
              'EEHEE': 'Rocklin.EEHEE.rfrdisparity.v1.1587.csv'}

stab2_df = pd.read_csv(stab_dir + stab2_file, usecols=['dataset','name','ec50_c','stabilityscore_c','ec50_t','stabilityscore_t'], comment='#')
stab_df = pd.merge(pd.read_csv(stab_dir + stab_file, usecols=['dataset','name','ec50_c','stabilityscore_c','ec50_t','stabilityscore_t'], comment='#'),
                  stab2_df, on=['dataset', 'name'], how='left',
                  suffixes=('_1','_2'))
meta_df = pd.merge(pd.read_csv(meta_dir + meta_file, usecols=['dataset','name','sequence', 'dssp'], comment='#'),
                   stab_df, on=['dataset', 'name'], how='left')

data_df = {}
for top in ['HHH', 'HEEH', 'EHEE', 'EEHEE']:
    data_df[top] = pd.merge(pd.read_csv(data_dir + data_files[top], comment='#'), meta_df, on=['dataset', 'name'], how='left')
    data_df[top] = pd.merge(pd.read_csv(data_dir + data_files[top], comment='#'), meta_df, on=['dataset', 'name'], how='left')
    data_df[top] = pd.merge(pd.read_csv(data_dir + data_files[top], comment='#'), meta_df, on=['dataset', 'name'], how='left')
    data_df[top] = pd.merge(pd.read_csv(data_dir + data_files[top], comment='#'), meta_df, on=['dataset', 'name'], how='left')

In [3]:
# extract most unexpectedly unstable designs and apply filters
sort_on = 'stability_diff'
rank_thresh = 50
rank_thresh_dict = {'HHH': rank_thresh, 'EHEE': rank_thresh, 'EEHEE': rank_thresh}
ec50_thresh = 1
stab_thresh = 0.3
pred_thresh = 1.0

# ------------------------------------------------------
out_df = pd.DataFrame()
for top in ['HHH', 'EHEE', 'EEHEE']:
    data_df[top].sort_values(sort_on, inplace=True)
    
    rank_thresh = rank_thresh_dict[top]
    
    # compute differences in ec50_c and ec50_t for v2 if available, otherwise use v1
    data_df[top]['ec50_diff_1'] = data_df[top]['ec50_t_1']-data_df[top]['ec50_c_1']
    data_df[top]['ec50_diff_2'] = data_df[top]['ec50_t_2']-data_df[top]['ec50_c_2']
    data_df[top].loc[data_df[top]['ec50_diff_2'].isna(), 'ec50_tc_diff'] = data_df[top]['ec50_diff_1'].abs()
    data_df[top].loc[~data_df[top]['ec50_diff_2'].isna(), 'ec50_tc_diff'] = data_df[top]['ec50_diff_2'].abs()

    # filter on both ec50_t/c_1/2, their consistency, and stability score 
    out_df = pd.concat([out_df,data_df[top].iloc[0:rank_thresh,:][(data_df[top].iloc[0:rank_thresh,:]['ec50_t_1'] < ec50_thresh) & 
                         ((data_df[top].iloc[0:rank_thresh,:]['ec50_t_2'] < ec50_thresh) | data_df[top].iloc[0:rank_thresh,:]['ec50_t_2'].isna()) &
                         (data_df[top].iloc[0:rank_thresh,:]['ec50_c_1'] < ec50_thresh) &
                         ((data_df[top].iloc[0:rank_thresh,:]['ec50_c_2'] < ec50_thresh) | data_df[top].iloc[0:rank_thresh,:]['ec50_c_2'].isna()) &
                         (data_df[top].iloc[0:rank_thresh,:]['stabilityscore_cnn_calibrated'] < stab_thresh) &
                         (data_df[top].iloc[0:rank_thresh,:]['stability_pred'] > pred_thresh)                                        ]])

In [4]:
comment_1 = '# Stability predictions made by RFR models (see '
for top in ['HHH', 'EHEE', 'EEHEE']:
    comment_1 = comment_1 + data_files[top] + '; '

with open('rd1_outlier_designs.csv', 'w', newline='') as f: 
    f.write('# Unexpectedly Unstable (Outlier) Designs\n')
    f.write(comment_1[:-2] + ')\n')
    f.write('# Top %d designs in each topology filtered using thresholds\n' %rank_thresh)
    f.write('# - ec_50 threshold < %f\n' %ec50_thresh)
    f.write('# - experimental stability threshold < %f\n' %stab_thresh)
    f.write('# - predicted stability threshold > %f\n' %pred_thresh)
    out_df.to_csv(f, sep=',')

In [5]:
out_df

,dataset,name,topology,group_index,stabilityscore_cnn_calibrated,stability_pred,stability_diff,num_nhbrs,num_nhbrs_gt,num_nhbrs_lt,...,stabilityscore_t_1,ec50_c_1,stabilityscore_c_1,ec50_t_2,stabilityscore_t_2,ec50_c_2,stabilityscore_c_2,ec50_diff_1,ec50_diff_2,ec50_tc_diff
2303,Rocklin,HHH_rd4_0821,HHH,1,-0.796107,1.183521,-1.979628,24.641,24.615,0.026,...,-0.230,-0.10,-0.45,0.357917,-0.086433,0.369595,-0.230852,0.150,-0.011678,0.011678
2300,Rocklin,HHH_rd4_0216,HHH,1,-0.121513,1.557339,-1.678853,25.727,25.633,0.094,...,0.030,-0.07,-0.53,0.184103,0.084128,0.026218,-0.480371,0.130,0.157885,0.157885
2294,Rocklin,HHH_rd4_0998,HHH,1,0.294450,1.961878,-1.667428,25.159,24.003,1.156,...,0.140,0.02,-0.25,0.213277,0.255922,0.221565,-0.154257,-0.050,-0.008288,0.008288
2301,Rocklin,HHH_rd4_0838,HHH,1,-0.227015,1.363691,-1.590706,24.616,24.507,0.109,...,-0.010,-0.11,-0.68,0.279891,-0.065850,0.432812,-0.419887,0.510,-0.152921,0.152921
2296,Rocklin,HHH_rd4_0628,HHH,1,0.109499,1.683769,-1.574270,24.982,23.475,1.507,...,-0.030,-0.04,-0.16,0.267183,0.138609,0.202980,-0.040977,-0.040,0.064203,0.064203
2290,Rocklin,HHH_rd1_0192,HHH,1,-0.359098,1.208268,-1.567365,25.070,24.844,0.226,...,-0.200,-0.18,-0.44,NaN,NaN,NaN,NaN,0.240,NaN,0.240000
2299,Rocklin,HHH_rd4_0068,HHH,1,-0.118895,1.447129,-1.566024,24.846,23.575,1.271,...,0.070,-0.08,-0.08,0.312959,0.250599,0.308448,0.102757,0.010,0.004512,0.004512
2292,Rocklin,HHH_rd1_0649,HHH,1,-0.286671,1.148335,-1.435007,26.626,25.988,0.638,...,-0.190,-0.02,-0.92,NaN,NaN,NaN,NaN,0.035,NaN,0.035000
2286,Rocklin,HHH_rd1_0501,HHH,1,-0.294126,1.092466,-1.386592,26.038,24.981,1.057,...,0.160,0.29,-0.31,NaN,NaN,NaN,NaN,0.110,NaN,0.110000
2280,Rocklin,HHH_rd2_0120,HHH,1,-0.189157,1.094453,-1.283610,26.602,25.488,1.114,...,-0.280,-0.07,-0.48,NaN,NaN,NaN,NaN,0.010,NaN,0.010000
